In [1]:
from room import Room
from person import Person
from action import Action
import metacode_helpers

In [2]:
import pandas as pd
import numpy as np

In [3]:
# initialise all actions
ALL_POSSIBLE_ACTIONS = metacode_helpers.initialise_all_actions()
ALL_CONVERSATION_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "conversation"]
ALL_ROOM_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "room"] 

In [4]:
# Create some rooms
HALL = Room("Hall")
KITCHEN = Room("Kitchen")
LOUNGE = Room("Lounge")

# Add adjacencies to the rooms (HALL is adjacent to KITCHEN and LOUNGE)
HALL.add_adjacent_room(KITCHEN)
HALL.add_adjacent_room(LOUNGE)

# Assign the Hall to be the entry to the house
HOUSE_ENTRY_ROOM = HALL 

ROOMS = [HALL, KITCHEN, LOUNGE]

In [5]:
from itertools import combinations

names = metacode_helpers.get_names(num_names=32)
np.random.seed(42)
ALL_PEOPLE = []
for personality_number in range(4):
    personality_vector = np.random.uniform(low = -1, high = 1, size = 5)
    initial_emotional_state_vector = np.random.uniform(low = -1, high = 1, size = 3)
    initial_location_state = np.random.choice(ROOMS)
    new_person = Person(name = names[personality_number], location_state=initial_location_state, all_possible_actions=ALL_POSSIBLE_ACTIONS, personality_vector=personality_vector, emotional_state_vector=initial_emotional_state_vector, conversation_partner=None)
    ALL_PEOPLE.append(new_person)

In [6]:
ALL_PEOPLE

[Robin, Patrick, Ronnie, Jacquelyn]

In [7]:
ALL_POSSIBLE_ACTIONS

array([shouts, shares, leaves_conversation, makes_smalltalk,
       makes_deeptalk, gives_controversial_opinion, changes_the_topic,
       shows_interest, shows_concern, laughs, jokes, gives_an_insult,
       brags, yawns, supports, rants, stands_in_corner, leaves_room,
       starts_conversation], dtype=object)

In [8]:
from collections import defaultdict

log = defaultdict(list)

for person in ALL_PEOPLE:
    print(person.get_name())
    log["person_before_action"].append(person.get_snapshot())
    if person.has_conversation_partner():
        log["conversation_partner_before_action"].append(person.get_conversation_partner().get_snapshot())

    # The person can start a conversation only if there's someone free to chat in their current room.
    available_room_act = ALL_ROOM_ACTIONS if person.get_location_state().is_someone_free_to_chat() else [action for action in ALL_ROOM_ACTIONS if action.get_name() != "starts_conversation"]

    # If in a conversation, the person can leave at any time.
    action = person.action_selection(available_conv_act=ALL_CONVERSATION_ACTIONS) if person.has_conversation_partner() else person.action_selection(available_room_act=available_room_act)
    
    # If in a conversation, update the person & conversation partner's emotional state according to the action
    if person.has_conversation_partner():
        # Change this person's emotional state by the GIVEN action emotional change vector
        person.set_emotional_state_vector(person.get_emotional_state_vector() + action.get_given_emotional_change_vector())
        # Change this person's conversation partner's emotional state by the RECEIVED action emotional change vector
        partner = person.get_conversation_partner()
        partner.set_emotional_state_vector(partner.get_emotional_state_vector() + action.get_received_emotional_change_vector())

        if action.get_name() == "leaves_conversation":
            # The conversation has ended, so both people now don't have conversation partners.
            person.set_conversation_partner(None)
            partner.set_conversation_partner(None)
            
    else: # Otherwise, deal with the room action
        if action.get_name() == "leaves_room":
            next_room = person.get_next_room()
            person.move_to_room(next_room)          
        if action.get_name() == "starts_conversation":
            # NOTE whether this is possible is handles by the action selection function
            # Find a person in the room who is free to chat
            partner = person.get_location_state().get_conversation_partner()
            person.set_conversation_partner(partner)
            partner.set_conversation_partner(person)
    break
    
    # log["action"] = action.get_snapshot()
    # log["person_after_action"] = person.get_snapshot()
    # log["conversation_partner_after_action"] = person.conversation_partner.get_snapshot()

Robin


In [9]:
log

defaultdict(list,
            {'person_before_action': [{'name': 'Robin',
               'personality_vector': array([-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272]),
               'emotional_state': array([-0.68801096, -0.88383278,  0.73235229]),
               'base_action_probs': {shouts: 0.07655987180318331,
                shares: 0.05214043044055023,
                leaves_conversation: 0.03229397311430043,
                makes_smalltalk: 0.05919391175178308,
                makes_deeptalk: 0.04895843300031232,
                gives_controversial_opinion: 0.03298589065217231,
                changes_the_topic: 0.03315694789892485,
                shows_interest: 0.045303135962283525,
                shows_concern: 0.04075600911988259,
                laughs: 0.07877462827221966,
                jokes: 0.0632321916336573,
                gives_an_insult: 0.03764411188021633,
                brags: 0.07655987180318331,
                yawns: 0.06892185772365128,